<a href="https://colab.research.google.com/github/Satyadeep-Dey/AI-experiments/blob/main/4_OpenAI_%2B_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chatbot that calls Open AI and functions | Google Colab **

We will develop a chatbot that utilizes OpenAI to respond to our questions and comments in chat, except when inquiries about the weather are made, in which case it will call a specific function.

In [ ]:
# Install necessary libraries
!pip install openai gradio --quiet

In [ ]:
# Import required libraries

import requests
import json
from openai import OpenAI
from google.colab import userdata
import gradio as gr

#define constants
MODEL = "gpt-4o-mini"

In [ ]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
print(openai_api_key)
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# Check if Open AI key has been set
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
# function that returns weather data from open-meteo.com API

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [ ]:
# you can call this tools , tools_weather and anything you like ..
# remember to use the same name when you call Open AI .
# this name will map to tools parameter

tools_weather = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
system_message = "You are a humorous chatbot ." # this is not mandatory

In [ ]:
# the function needs to have this name and signature
def chat(message, history):

    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    #messages = history + [{"role": "user", "content": message}] -> even this will work if you don't want system message

    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools_weather, # this "tools_weather" has been defined in previous cell of type function ...
        tool_choice="auto" # Model decides when to call the tool
    )

    if completion.choices[0].finish_reason=="tool_calls":
        # If tool is called
        tool_call = completion.choices[0].message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        result = get_weather(args["latitude"], args["longitude"])
        messages.append(completion.choices[0].message)  # append model's function call message
        messages.append({                               # append result message
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result)
            })


    completion_2 = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools_weather)

    return completion_2.choices[0].message.content


In [ ]:
# now let's use Gradio to make a chat interface with just a line of code !!
gr.ChatInterface(fn=chat, type="messages").launch(pwa=True,inbrowser=True,share=True)

#To show errors in colab notebook, set debug=True in launch()